In [ ]:
import librosa 
import numpy as np
import matplotlib.pyplot as plt

import steme as st

import IPython.display as ipd

In [ ]:
librosa.util.list_examples()

In [ ]:
x, fs = librosa.load(librosa.example("brahms"))
# you can explore further examples if you want to:
# x, fs = librosa.load(librosa.example("sweetwaltz"))

In [ ]:
ipd.Audio(x, rate=fs)

# linear axis versus log axis

In [ ]:
linear_axis = np.arange(30,600,1)
log_axis = st.dataset.variables_non_linear()
log_axis = log_axis[log_axis < 350]

In [ ]:
def plot_comparison(T, t, freqs, ttypes, subplot_titles, fig_title=None):
    """
    helper function to plot tempograms side-by-side.
    """
    figsize = (15, 5)
    num_tempograms = len(T)
    fig, ax = plt.subplots(1, num_tempograms, figsize=figsize)

    for idx in range(num_tempograms):
        kwargs = st.utils._tempogram_kwargs(t[idx], freqs[idx])

        ax[idx].imshow(T[idx], **kwargs)

        xlim = (t[idx][0], t[idx][-1])
        ylim = (freqs[idx][0], freqs[idx][-1])

        plt.setp(ax, xlim=xlim, ylim=ylim)
        
        if ttypes[idx] == "log":
            labels = [item.get_text() for item in ax[0].get_yticklabels()]
            new_labels = np.rint(log_axis[::20]).astype(int)
            ax[idx].set_yticklabels(new_labels)

        if fig_title is not None:
            fig.suptitle(fig_title, fontsize=16)

        ax[idx].set_xlabel("Time (s)")
        ax[idx].set_ylabel("Tempo (BPM)")
        ax[idx].title.set_text(subplot_titles[idx])
    return fig, ax

In [ ]:
linear_fT, linear_ft, linear_ffreqs = st.audio.tempogram(x=x, sr=fs, window_size_seconds=10, t_type="fourier", theta=linear_axis)
fT, ft, ffreqs = st.audio.tempogram(x=x, sr=fs, window_size_seconds=10, t_type="fourier", theta=log_axis)

fig, ax = plot_comparison(
    T=[linear_fT, fT], 
    t=[linear_ft, ft], 
    freqs=[linear_ffreqs, ffreqs], 
    subplot_titles=["linear axis", "logarithmic axis"],
    ttypes=["linear", "log"],
    fig_title="Fourier tempogram"
)

In [ ]:
linear_aT, linear_at, linear_afreqs = st.audio.tempogram(x=x, sr=fs, window_size_seconds=10, t_type="autocorrelation", theta=linear_axis)
aT, at, afreqs = st.audio.tempogram(x=x, sr=fs, window_size_seconds=10, t_type="autocorrelation", theta=log_axis)

fig, ax = plot_comparison(
    T=[linear_aT, aT], 
    t=[linear_at, at], 
    freqs=[linear_afreqs, afreqs], 
    subplot_titles=["linear axis", "logarithmic axis"],
    ttypes=["linear", "log"],
    fig_title="Autocorrelation tempogram"
)

In [ ]:
linear_hT, linear_ht, linear_hfreqs = st.audio.tempogram(x=x, sr=fs, window_size_seconds=10, t_type="autocorrelation", theta=linear_axis)
hT, ht, hfreqs = st.audio.tempogram(x=x, sr=fs, window_size_seconds=10, t_type="hybrid", theta=log_axis)

fig, ax = plot_comparison(
    T=[linear_hT, hT], 
    t=[linear_ht, ht], 
    freqs=[linear_hfreqs, hfreqs], 
    subplot_titles=["linear axis", "logarithmic axis"],
    ttypes=["linear", "log"],
    fig_title="Hybrid tempogram"
)

# same representation for steady and changing tempo

In [ ]:
x, x_fs = librosa.load(librosa.example("brahms"))
y, y_fs = librosa.load(librosa.example("choice"))

In [ ]:
x_fT, x_ft, x_ffreqs = st.audio.tempogram(x=x, sr=x_fs, window_size_seconds=10, t_type="fourier", theta=log_axis)
y_fT, y_ft, y_ffreqs = st.audio.tempogram(x=y, sr=y_fs, window_size_seconds=10, t_type="fourier", theta=log_axis)

fig, ax = plot_comparison(
    T=[x_fT, y_fT], 
    t=[x_ft, y_ft], 
    freqs=[x_ffreqs, y_ffreqs], 
    ttypes=["log", "log"],
    subplot_titles=["changing tempo (brahms)", "steady tempo (choice)"],
    fig_title="Fourier tempogram"
)

In [ ]:
x_fT, x_ft, x_ffreqs = st.audio.tempogram(x=x, sr=x_fs, window_size_seconds=10, t_type="autocorrelation", theta=log_axis)
y_fT, y_ft, y_ffreqs = st.audio.tempogram(x=y, sr=y_fs, window_size_seconds=10, t_type="autocorrelation", theta=log_axis)

fig, ax = plot_comparison(
    T=[x_fT, y_fT], 
    t=[x_ft, y_ft], 
    freqs=[x_ffreqs, y_ffreqs], 
    ttypes=["log", "log"],
    subplot_titles=["changing tempo (brahms)", "steady tempo (choice)"],
    fig_title="Autocorrelation tempogram"
)

In [ ]:
x_fT, x_ft, x_ffreqs = st.audio.tempogram(x=x, sr=x_fs, window_size_seconds=10, t_type="hybrid", theta=log_axis)
y_fT, y_ft, y_ffreqs = st.audio.tempogram(x=y, sr=y_fs, window_size_seconds=10, t_type="hybrid", theta=log_axis)

fig, ax = plot_comparison(
    T=[x_fT, y_fT], 
    t=[x_ft, y_ft], 
    freqs=[x_ffreqs, y_ffreqs],
    ttypes=["log", "log"],
    subplot_titles=["changing tempo (brahms)", "steady tempo (choice)"],
    fig_title="Hybrid tempogram"
)


In [ ]:
increasing_tempo = np.array([])
sr = 22500
for i in range(1,8):
    tmp = st.audio.click_track(bpm=100*i, sr=sr, duration=3)
    increasing_tempo = np.append(increasing_tempo, tmp)
    
x_fT, x_ft, x_ffreqs = st.audio.tempogram(x=increasing_tempo, sr=sr, window_size_seconds=10, t_type="fourier", theta=linear_axis)
y_fT, y_ft, y_ffreqs = st.audio.tempogram(x=increasing_tempo, sr=sr, window_size_seconds=10, t_type="fourier", theta=log_axis)

fig, ax = plot_comparison(
    T=[x_fT, y_fT], 
    t=[x_ft, y_ft], 
    freqs=[x_ffreqs, y_ffreqs], 
    subplot_titles=["linear", "log"],
    ttypes=["linear", "log"],
    fig_title="fourier tempogram"
)


# Interactive view

In [ ]:
import holoviews as hv 
import panel as pn
hv.extension("bokeh", logo=False)

In [ ]:
increasing_tempo = np.array([]) #np.zeros([fs*7*3])
sr = 22500
for i in range(1,8):
    tmp = audio.click_track(bpm=50*i, sr=sr, duration=3)
    increasing_tempo = np.append(increasing_tempo, tmp)

In [ ]:
audio_data = np.int16(increasing_tempo * 32767)

In [ ]:
ipd.Audio(increasing_tempo, rate=sr)

## Fourier

In [ ]:
fT, ft, ffreqs = audio.tempogram(x=increasing_tempo, sr=sr, window_size_seconds=10, t_type="fourier", theta=log_axis)
st.utils.plot_tempogram(fT, ft, ffreqs)

In [ ]:
fT, ft, ffreqs = st.audio.tempogram(x=increasing_tempo, sr=sr, window_size_seconds=10, t_type="fourier", theta=linear_axis)

spec_gram = hv.Image((ft, ffreqs, fT), ["Time (s)", "Tempo (BPM)"]).opts(width=600)
audio = pn.pane.Audio(audio_data, sample_rate=sr, name='Audio', throttle=500)

def update_playhead(x,y,t):
    if x is None:
        return hv.VLine(t)
    else:
        audio.time = x
        return hv.VLine(x)

tap_stream = hv.streams.SingleTap(transient=True)
time_play_stream = hv.streams.Params(parameters=[audio.param.time], rename={'time': 't'})
dmap_time = hv.DynamicMap(update_playhead, streams=[time_play_stream, tap_stream])
out = pn.Column( audio, 
               (spec_gram * dmap_time))

In [ ]:
out

## Autocorrelation

In [ ]:
spec_gram = hv.Image((at, afreqs, aT), ["Time (s)", "Tempo (BPM)"]).opts(width=600)
audio = pn.pane.Audio(audio_data, sample_rate=fs, name='Audio', throttle=500)

def update_playhead(x,y,t):
    if x is None:
        return hv.VLine(t)
    else:
        audio.time = x
        return hv.VLine(x)

tap_stream = hv.streams.SingleTap(transient=True)
time_play_stream = hv.streams.Params(parameters=[audio.param.time], rename={'time': 't'})
dmap_time = hv.DynamicMap(update_playhead, streams=[time_play_stream, tap_stream])
out = pn.Column( audio, 
               (spec_gram * dmap_time))

In [ ]:
out

## Hybrid

In [ ]:
spec_gram = hv.Image((ht, hfreqs, hT), ["Time (s)", "Tempo (BPM)"]).opts(width=600)
audio = pn.pane.Audio(audio_data, sample_rate=fs, name='Audio', throttle=500)

def update_playhead(x,y,t):
    if x is None:
        return hv.VLine(t)
    else:
        audio.time = x
        return hv.VLine(x)

tap_stream = hv.streams.SingleTap(transient=True)
time_play_stream = hv.streams.Params(parameters=[audio.param.time], rename={'time': 't'})
dmap_time = hv.DynamicMap(update_playhead, streams=[time_play_stream, tap_stream])
out = pn.Column( audio, 
               (spec_gram * dmap_time))

In [ ]:
out